In [29]:
import pandas as pd
from pygeocoder import Geocoder
from time import sleep
import numpy as np

In [6]:
df = pd.read_csv('02_df.csv')
df.columns

Index(['Unnamed: 0', 'Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)',
       'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula',
       'href', 'ts_year', 'ts_month', 'ts_day', 'ts_time', 'ts_date'],
      dtype='object')

In [7]:
df['loc_str'] = df['Location'] + " " + df['Area'] + " "+ df['Country']

In [8]:
df['loc_str'].head()

0                  Manuelita Cocos Island COSTA RICA
1                  Manuelita Cocos Island COSTA RICA
2                                                NaN
3    Stillwater Cove, Monterey County California USA
4        Floridana Beach, Brevard County Florida USA
Name: loc_str, dtype: object

In [9]:
len(df)

1342

In [20]:
results = []
for i, l in enumerate(df['loc_str']):
    if i % 10 == 0:
        print('Record {}, sleeping 10 seconds...'.format(i))
        sleep(10)
    
    res_dict = {'loc_str': l}
    if l == 'NaN' or l == 'nan':
        results.append(res_dict)
        continue
    try:
        res = Geocoder.geocode(l)
    except:
        results.append(res_dict)
        continue
    try:
        coords = res[0].coordinates
        res_dict['lat'] = coords[0]
        res_dict['long'] = coords[1]
    except:
        pass
    try:
        res_dict['geo_country'] = res[0].country
    except:
        pass
    results.append(res_dict)

Record 0, sleeping 10 seconds...
Record 10, sleeping 10 seconds...
Record 20, sleeping 10 seconds...
Record 30, sleeping 10 seconds...
Record 40, sleeping 10 seconds...
Record 50, sleeping 10 seconds...
Record 60, sleeping 10 seconds...
Record 70, sleeping 10 seconds...
Record 80, sleeping 10 seconds...
Record 90, sleeping 10 seconds...
Record 100, sleeping 10 seconds...
Record 110, sleeping 10 seconds...
Record 120, sleeping 10 seconds...
Record 130, sleeping 10 seconds...
Record 140, sleeping 10 seconds...
Record 150, sleeping 10 seconds...
Record 160, sleeping 10 seconds...
Record 170, sleeping 10 seconds...
Record 180, sleeping 10 seconds...
Record 190, sleeping 10 seconds...
Record 200, sleeping 10 seconds...
Record 210, sleeping 10 seconds...
Record 220, sleeping 10 seconds...
Record 230, sleeping 10 seconds...
Record 240, sleeping 10 seconds...
Record 250, sleeping 10 seconds...
Record 260, sleeping 10 seconds...
Record 270, sleeping 10 seconds...
Record 280, sleeping 10 seconds

In [21]:
len(results)

1342

In [22]:
geocoded = pd.DataFrame(results)
geocoded.head()

geo_country        lat                                          loc_str  \
0     Costa Rica   5.528228                Manuelita Cocos Island COSTA RICA   
1     Costa Rica   5.528228                Manuelita Cocos Island COSTA RICA   
2       Thailand  18.775632                                              NaN   
3  United States  36.563945  Stillwater Cove, Monterey County California USA   
4  United States  27.945631      Floridana Beach, Brevard County Florida USA   

         long  
0  -87.057430  
1  -87.057430  
2  100.773042  
3 -121.942334  
4  -80.497996

In [23]:
geocoded.to_csv('geocoded.csv', index=False)

In [24]:
geocoded.rename(columns={'loc_str': 'geo_loc_str'}, inplace=True)

In [25]:
df_geo = pd.concat([df, geocoded], axis=1)
df_geo.head()

Unnamed: 0   Case Number                  Date    Year          Type  \
0           0  2017.11.30.b           30-Nov-2017  2017.0    Unprovoked   
1           1  2017.11.30.a           30-Nov-2017  2017.0    Unprovoked   
2           2  2017.11.25.R  Reported 25-Nov-2017  2017.0  Sea Disaster   
3           3    2017.11.24           24-Nov-2017  2017.0    Unprovoked   
4           4    2017.11.18           18-Nov-2017  2017.0    Unprovoked   

      Country          Area                          Location  \
0  COSTA RICA  Cocos Island                         Manuelita   
1  COSTA RICA  Cocos Island                         Manuelita   
2       LIBYA           NaN                    Gars Garabulli   
3         USA    California  Stillwater Cove, Monterey County   
4         USA       Florida   Floridana Beach, Brevard County   

           Activity             Name     ...     ts_year ts_month ts_day  \
0      Scuba Diving       __ Jimenez     ...        2017       11     30   
1      Scuba diving  Rohina Bhandari     ...        2017       11     30   
2  2 boats capsized      31 migrants     ...        2017       11     25   
3      Spearfishing   Grigor Azatian     ...        2017       11     24   
4           Surfing    Kaia Anderson     ...        2017       11     18   

  ts_time     ts_date                                          loc_str  \
0     NaN  2017-11-30                Manuelita Cocos Island COSTA RICA   
1     NaN  2017-11-30                Manuelita Cocos Island COSTA RICA   
2     NaN  2017-11-25                                              NaN   
3   13:45  2017-11-24  Stillwater Cove, Monterey County California USA   
4     NaN  2017-11-18      Floridana Beach, Brevard County Florida USA   

     geo_country        lat                                      geo_loc_str  \
0     Costa Rica   5.528228                Manuelita Cocos Island COSTA RICA   
1     Costa Rica   5.528228                Manuelita Cocos Island COSTA RICA   
2       Thailand  18.775632                                              NaN   
3  United States  36.563945  Stillwater Cove, Monterey County California USA   
4  United States  27.945631      Floridana Beach, Brevard County Florida USA   

         long  
0  -87.057430  
1  -87.057430  
2  100.773042  
3 -121.942334  
4  -80.497996  

[5 rows x 30 columns]

In [27]:
len(df_geo[df_geo['loc_str'] != df_geo['geo_loc_str']])

106

In [ ]:
# Nan values were coded as Thailand

In [32]:
len(df_geo[df_geo['geo_loc_str'].isnull()])

106

In [33]:
geocoded.loc[geocoded['geo_loc_str'].isnull(), 'lat'] = None
geocoded.loc[geocoded['geo_loc_str'].isnull(), 'long'] = None
df_geo.loc[df_geo['geo_loc_str'].isnull(), 'lat'] = None
df_geo.loc[df_geo['geo_loc_str'].isnull(), 'long'] = None

In [34]:
df_geo.to_csv('02_df_geo.csv', index=False)
geocoded.to_csv('geocoded.csv', index=False)